# MO651 - Robótica Móvel - Trabalho 01
### Equipe: 
* Carlos Victor - RA: 230261
* Daiane Mendes - RA: 230210
* João Paulo - RA: 230221
* Pedro Olímpio - RA: 230256


## Introdução
Em diversos cenários dentro da área de estudo da Robótica Móvel são necessários comportamento autônomos dos robôs. Neste trabalho apresentamos técnicas de evitar colisão e imobilização do robô com obstáculos (*Avoid Obstacles*), técnicas de exploração do ambiente mantendo a rota paralela e próxima das paredes do ambiente (*Wall Follow*) e técnicas de alcance de pontos específicos do cenário (*Go to Goal*).

As técnicas implementadas foram executadas em um ambiente simulado usando o simulador v-rep com o robô Pioneer 3-DX.

## Metodologia
As seguir descreveremos a métodologia utilizada desenvolver os sistemas propostos.

### Célula para inclusão de bibliotecas
Na célula a seguir são incluidas as bibliotecas utilizadas no trabalho. A biblioteca *skfuzzy* é utilizada para operações em lógica fuzzy. A biblioteca *robot* no diretório "../src" faz a integração com a simulação.

In [1]:
import sys, time, random, math, cv2
import numpy as np
from matplotlib import pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl

sys.path.insert(0, '../src')
from robot import Robot

### Cálculo da Odometria

In [2]:
def calculate_delta(begin, end, spinOrient):
    if math.isclose(float(end), float(begin), abs_tol=0.00001):
        return 0
    end = (limit + end) % limit
    begin = (limit + begin) % limit

    ans = abs(end - begin)
    if spinOrient:
        if begin < end:
            return limit - ans
        return ans
    if begin > end:
        return limit - ans
    return ans

def calculate_speed(time, clockwiseSpin, lastEncoder, motorHandle):
    encoder = robot.get_joint_position(motorHandle)
    delta = calculate_delta(lastEncoder, encoder, clockwiseSpin)
    
    lastEncoder = encoder
    if delta > math.pi:
        delta = (2 * math.pi) - delta
        clockwiseSpin = not clockwiseSpin
    
    speed = 0.0975 * (delta/time)
    if(clockwiseSpin):
        return -speed, clockwiseSpin, lastEncoder
    
    return speed, clockwiseSpin, lastEncoder

def to180Universe(alpha):
    alpha = alpha % (2 * math.pi)
    if alpha > math.pi:
        return alpha - (2 * math.pi)
    return alpha

def to360Universe(alpha):
    if alpha < 0:
        return (2 * math.pi) + alpha
    return alpha

def add_angle(alpha, beta):
    angle = to360Universe(alpha) + to360Universe(beta)
    return to180Universe(angle)

def new_position(alpha, x, y):
    global lastTime
    global lastEncoderL, lastEncoderR
    global clockwiseSpinL, clockwiseSpinR
    
    now = time.time()
    dtime = now-lastTime
    lastTime = now
    
    vL, clockwiseSpinL, lastEncoderL = calculate_speed(dtime, clockwiseSpinL, lastEncoderL, 'left')
    vR, clockwiseSpinR, lastEncoderR = calculate_speed(dtime, clockwiseSpinR, lastEncoderR, 'right')
    
    w = (vR-vL)/0.36205
    v = (vR + vL)/2.0
    
    dAlpha = w*dtime
    dL = vL*dtime
    dR = vR*dtime
    dM = (dL+dR)/2.0
    dX = dM*math.cos(add_angle(alpha, dAlpha/2.0))
    dY = dM*math.sin(add_angle(alpha, dAlpha/2.0))
    return x+dX, y+dY, add_angle(alpha, dAlpha)

def plot_odometry_vs_GT(path, path_odometry):
    _, ax = plt.subplots(figsize=(9, 9))
    ax.plot(path_odometry[:,0], path_odometry[:,1], label = 'Odometry')
    ax.plot(path[:,0], path[:,1], label = 'Ground Truth')
    ax.grid(True)
    ax.legend(loc='best')
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title('Odometry x Ground Truth')
    plt.savefig('odometry'+'.png') 
#     plt.show()

### Go to Goal com PID
Essa classe foi definida para direcionar o robô para uma posição objetivo, usando o controlador PID (Proportional + Integral + Derivative).  A ideia é que dado a posição atual do robô, o ângulo atual e a posição objetivo, será calculado uma velocidade para a roda esquerda e para roda direita, de modo a controlar e direcionar o robô para a posição objetivo.

Inicialmente são definidas constantes de limiar de erro e dos coeficientes proporcional (Kp), integral (Ki) e derivado (Kd). Além disso, são declaradas variáveis para guardar o erro acumulado, o erro atual e o erro anterior. Foram declarados dois tipos de erros e dois tipos de coeficientes, um de acordo com a posição e outro com o ângulo.

Ao chamar a função go_to_goal_PID, passando a posição atual do robô, o ângulo atual e a posição objetivo. De início, é checado se o erro do ângulo é menor que o limiar e em caso afirmativo, é calculado o novo error angular e se o robô ainda estiver distante da posição objetivo é chamado a função para calcular a nova velocidade das rodas esquerda e direita, considerando as variáveis de posição. Caso contrário, é realizado o cálculo da nova velocidade considerando as variáveis do ângulo. Em ambos os métodos a ideia seguida é do modelo de controlador PID (visto em aula). 

Um esquema do modelo de controlador PID pode ser visto na imagem a seguir. A ideia é que o controlador PID leva em conta o que aconteceu, o que está acontecendo e o que provavelmente acontecerá, e combina essas coisas para produzir o novo valor para controlar o robô.

<img src="../imgs/PID.png">

A imagem a seguir mostra um exemplo de uma rota tomada pelo robô para chegar no destino (vaso com plantas).

<img src="../imgs/goToGoalPID.png">



In [3]:
class GoToGoalPID():
    epsilonA = 25
    KpA = 1.5
    KiA = KpA / 10.0
    KdA = KpA * 5
    epsilonP = 0.01
    KpP = 0.7
    KdP = KpP * 5
    
    def __init__(self):
        self.errorA = self.epsilonA
        self.oldErrorA = 0
        self.accErrorA = 0
        self.errorP = self.epsilonP
        self.oldErrorP = 0
    
    def calcule_norm_angle(self, angle):
        return angle % (2*math.pi)

    def calcule_x1_y1_angle(self, position, angle, goal):
        x1 = position[0] - goal[0]
        y1 = position[1] - goal[1]
        angle = self.calcule_norm_angle(angle)
        return x1, y1, angle

    def calcule_errorA(self, position, angle, goal):
        x1, y1, angle = self.calcule_x1_y1_angle(position, angle, goal)
        goalAngle = self.calcule_norm_angle(math.pi + math.atan2(y1, x1))
        self.errorA = math.atan2(math.sin(goalAngle-angle), math.cos(goalAngle-angle))

    def calcule_go_to_position(self, position, angle, goal):
        x1, y1, angle = self.calcule_x1_y1_angle(position, angle, goal)
        goalAngle = self.calcule_norm_angle(math.pi + math.atan2(y1, x1))
        deltaA = math.degrees(math.atan2(math.sin(goalAngle-angle), math.cos(goalAngle-angle)))
        sig = -1 if abs(deltaA) > 100 else 1

        self.errorP = sig * math.sqrt( ((goal[0]-position[0])**2) + ((goal[1]-position[1])**2) )
        diffErrorP = self.errorP - self.oldErrorP
        u = (self.KpP * self.errorP) + (self.KdP * diffErrorP)
        self.oldErrorP = self.errorP
        return [u,u]


    def calcule_go_to_angle(self, position, angle, goal):
        self.calcule_errorA(position, angle, goal)
        self.accErrorA = self.accErrorA + self.errorA
        diffErrorA = self.errorA - self.oldErrorA
        u = (self.KpA * self.errorA) + (self.KiA * self.accErrorA) + (self.KdA * diffErrorA)
        self.oldErrorA = self.errorA
        return [u,-u]


    def go_to_goal_PID(self, position, angle, goal):
        if abs(math.degrees(self.errorA)) < self.epsilonA:
            self.calcule_errorA(position, angle, goal)
            if abs(self.errorP) >= self.epsilonP:
                return self.calcule_go_to_position(position, angle, goal)
            else:
                return [0,0]
        else:
            return self.calcule_go_to_angle(position, angle, goal)



### Wall Follow com Fuzzy

A função followWall é responsável por fazer o robô movimentar-se seguindo as paredes do cenário utilizando a lógica Fuzzy.

A ideia desta função é fazer o robô reagir a três tipos de situações diferentes. A primeira delas, é quando o robô não possui nenhum objeto no seu campo de sensores. Neste caso, o robô segue em frente, até que seu sensor frontal capture algum objeto (assumindo como uma parede). Posteriormente, estando muito próximo de algum objeto com seu sensor frontal, o robô vira à esquerda até seu sensor frontal estar livre e, depois disso, segue em frente. Ademais, quando o robô chega ao final da lateral da parede que está sendo percorrida, dois casos podem acontecer:

**Caso 1**: existe uma outra parede em frente ao robô. Neste caso, ele deve virar novamente à esquerda e seguir em frente;

<img src="../imgs/caso1.jpg">

**Caso 2**: não existe uma outra parede em frente ao robô. Neste caso, a parede que está sendo percorrida continua à direita e, por esse motivo, o robô deve virar à direita até que seu sensor frontal capture levemente a presença desta nova parede, para que ele possa continuar o seu destino.

<img src="../imgs/caso2.jpg">

Definições para a lógica Fuzzy.

**Antecedentes**: o distRight é o antecedente responsável por elencar a distância do sensor 6 do robô. Ele possui 4 níveis de distância, que são: very-close, quando o sensor está muito próximo de algum obstáculo; close, quando o robô encontra-se em uma distância relativamente próxima, com possibilidade de colisão; medium, que indica que possivelmente o robô está se aproximando de um obstáculo; away, que indica que não há nenhum obstáculo próximo ao robô.

**Consequentes**: três tipos diferentes de ação podem ser tomadas pelo robô, de acordo com o movimento que ele precisa fazer. No caso que ele deve seguir em frente, ambas as velocidades de suas rodas devem estar positivas e iguais. No caso que ele precisa virar à esquerda, sua roda esquerda deve estar com uma velocidade negativa e sua roda direita deve estar com o módulo desta velocidade. Finalmente, no caso em que o robô vira à direita, a velocidade da roda direita deve ser a metade da roda esquerda, pois o robô não pode parar de andar para frente para realizar a curva que permite que ele continue acompanhando a parede.

Por esse motivo, seis consequentes foram definidos. Quatro são responsáveis pelas velocidades das rodas, tanto positivas quanto negativas, para os casos em que o robô deve seguir reto ou virar à esquerda. Os outros dois consequentes, são definidos de modo a colocar a velocidade da roda direita com a metade da velocidade da roda da esquerda, para contemplar o caso faltante.

O conjunto de regras está definido com três regras principais, e são aplicadas duas vezes: uma vez para a roda esquerda e outra vez para a roda esquerda.

A primeira regra seta a velocidade da roda esquerda como negativa e a velocidade da roda direita como positiva, para, deste modo, o robô virar à esquerda. Ela acontece quando o sensor "right sensor" está com o atributo "very-close". A segunda regra faz o robô realizar a curva à direita. Elá é acionada quando o robô não captura nada a sua frente e também quando o sensor da direita está captando a parede. Finalmente, a terceira regra coloca a velocidade das duas rodas como positivas, para o robo andar para frente, quando ambos os sensores não estão captando nenhum objeto próximo.



In [4]:
def followWall(rightSensor, auxSensor):
    defaultVelocity = 1
    minDist = 0.0
    maxDist = 6.0
    interval = 0.1

    # Definicao dos antecedentes
    distAux = ctrl.Antecedent(np.arange(minDist, maxDist, interval), 'aux dist')
    distRight = ctrl.Antecedent(np.arange(minDist, maxDist, interval), 'right dist')

    # Definicao dos consequentes
    velLeftWheel = ctrl.Consequent(np.arange(-defaultVelocity, defaultVelocity + 1.0, interval), 'vel left wheel')
    velRightWheel = ctrl.Consequent(np.arange(-defaultVelocity, defaultVelocity + 1.0, interval), 'vel right wheel')

    distRight['very-close'] = fuzz.trimf(distRight.universe, [0.0, 0.2, 0.4])
    distRight['close'] = fuzz.trimf(distRight.universe, [0.3, 0.5, 0.7])
    distRight['medium'] = fuzz.trimf(distRight.universe, [0.6, 1.0, 1.4])
    distRight['away'] = fuzz.trimf(distRight.universe, [1.3, 6.0, 6.0])

    distAux['very-close'] = fuzz.trimf(distAux.universe, [0, 0.2, 0.4])
    distAux['close'] = fuzz.trimf(distAux.universe, [0.3, 0.5, 0.7])
    distAux['medium'] = fuzz.trimf(distAux.universe, [0.6, 1.0, 1.4])
    distAux['away'] = fuzz.trimf(distAux.universe, [1.3, 6.0, 6.0])

    velLeftWheel['positive'] = fuzz.trimf(velLeftWheel.universe, [0.0, defaultVelocity, defaultVelocity])
    velLeftWheel['negative'] = fuzz.trimf(velLeftWheel.universe, [-defaultVelocity, -defaultVelocity, 0.0])

    velRightWheel['positive'] = fuzz.trimf(velRightWheel.universe, [0.0, defaultVelocity, defaultVelocity])
    velRightWheel['negative'] = fuzz.trimf(velRightWheel.universe, [-defaultVelocity, -defaultVelocity, 0.0])

    velLeftWheel['positive-right'] = fuzz.trimf(velLeftWheel.universe, [0.0, defaultVelocity, defaultVelocity])
    velRightWheel['positive-right'] = fuzz.trimf(velRightWheel.universe, [-defaultVelocity+0.6, -defaultVelocity+0.6, 0.0])

    # Regras
    rules = {}

    rules['turn left1'] = ctrl.Rule(distRight['very-close'], velLeftWheel['negative'])
    rules['turn left2'] = ctrl.Rule(distRight['very-close'], velRightWheel['positive'])

    rules['turn right1'] = ctrl.Rule(distRight['away'] & (distAux['very-close'] | distAux['close']),
                                     velLeftWheel['positive-right'])
    rules['turn right2'] = ctrl.Rule(distRight['away'] & (distAux['very-close'] | distAux['close']),
                                                          velRightWheel['positive-right'])

    rules['forward1'] = ctrl.Rule((distRight['close']
                                      | distRight['medium']
                                      | distRight['away']), velRightWheel['positive'])
    rules['forward2'] = ctrl.Rule((distRight['close']
                                       | distRight['medium']
                                       | distRight['away']), velLeftWheel['positive'])

    runSimulation = ctrl.ControlSystem([rules[key] for key in rules])
    startSimulation = ctrl.ControlSystemSimulation(runSimulation)

    startSimulation.input['right dist'] = rightSensor
    startSimulation.input['aux dist'] = auxSensor

    startSimulation.compute()

    return startSimulation.output['vel left wheel'], startSimulation.output['vel right wheel']


### Go to Goal com Fuzzy

A função goToGoal foi implementada utilizando lógica Fuzzy e será responsável pela movimentação do robô a partir do eu ponto inicial até um ponto destino no cenário. 

A ideia desta função é fazer o robô rotaciona em direção ao ponto destino e ir seguindo em frente naquela direção, podendo efetuar algumas rotações de menor velocidade ao longo do caminho a ser percorrido.

As definições para a lógica Fuzzy que foram feitas são as seguintes:

**Antecedentes**: O 'distance' é o antecedente responsável por elencar a distância da localização atual do robô para o ponto objetivo. Ele possui 4 níveis de distância, que são: 'none', quando robô está no ponto destino; 'small', quando o robô encontra-se em uma distância relativamente próxima do ponto objetivo; 'medium', que indica uma distância maior que 'small'; 'big', indica uma distância grande, podendo assim aumentar a velocidade do robô ao máximo.

O 'angular distance' é o antecedente responsável por elencar o valor de rotação que deve ser feito para o robô estar na direção correta do ponto objetivo. Possui 4 níveis de rotação: 'small from left', que indica uma rotação a esquerda com movimento em ambas as rodas; 'smaller from left' se trata de uma rotação mais lenta e sutil à esquerda, com movimento apenas em uma roda. 'small from right' e 'smaller from right' se tratam das mesmas definições, agora aplicadas para a direta, ao invés de esquerda. 

**Consequentes**: as ações a serem tomadas se tratam da velocidade que será aplicada em cada roda do robô. Aumentar a velocidade em uma das rodas e reduzir na outra faz com que ele efetue uma curva mais sutil. Parar uma roda e setar uma velocidade positiva na outra roda fará o robô girar no próprio eixo. Manter as duas rodas com mesma velocidade faz com que o robô siga reto em uma direção. 

O conjunto de regras está definido com três conjuntos de regras principais que se aplicam as rodas direita e esquerda, simultaneamente.

A primeira regra seta a velocidade das rodas esquerda e direita como zero, ou seja, o robô atingiu seu destino. As regras dois e três são utilizadas para distâncias longas do ponto atual do robô para o objetivo, onde o mesmo irá efetuar curvas com velocidade em ambas as rodas (maior velocidade na roda inversa á direção da curva). Regras quatro e cinco se tratam de rotação no próprio eixo do robô, ou seja, uma roda parada e outra com velocidade baixa, essa regra se aplica predominantemente quando o robô está bem próximo na posição objetivo. As regras seis e sete são para seguir o objetivo, se o robô estiver distante e na direção correta, pode-se adicionar velocidade máxima em ambas as rodas, enquanto se a distâncias do objetivo for menor, é necessário que haja uma diminuição da velocidade.  



In [5]:
class GoToGoalFuzzy():
    def __init__(self):
        ## Antecedents ##
        # Distance in meters
        distance = ctrl.Antecedent(np.arange(0, 20, 0.5), 'distance')

        # Angular distance
        angular_distance = ctrl.Antecedent(np.arange(0, 360, 1), 'angular distance') 

        ## Consequents ##
        # Velocity on the right wheel
        right_wheel = ctrl.Consequent(np.arange(0, 4, 0.5), 'right wheel speed')
        # Velocity on the left wheel
        left_wheel = ctrl.Consequent(np.arange(0, 4, 0.5), 'left wheel speed') 

        ## Memberships distance functions ##
        distance['none'] = fuzz.trimf(distance.universe, [0, 0, 0.15])
        distance['small'] = fuzz.trimf(distance.universe, [0, 0.2, 0.8])
        distance['medium'] = fuzz.trapmf(distance.universe, [0.7, 1.5, 2, 3])
        distance['big'] = fuzz.trapmf(distance.universe, [2.5, 5, 20, 20])

        ## Angular ##
        angular_distance['small from left'] = fuzz.trimf(angular_distance.universe, [0, 0, 15]) # diferença angular pequena pela esquerda
        angular_distance['smaller from left'] = fuzz.trapmf(angular_distance.universe, [0, 0, 179, 180])
        angular_distance['smaller from right'] = fuzz.trapmf(angular_distance.universe, [180, 181, 359, 360])
        angular_distance['small from right'] = fuzz.trimf(angular_distance.universe, [345, 360, 360])

        ## Right wheel ##
        right_wheel['zero'] = fuzz.trimf(right_wheel.universe, [0, 0, 0])
        right_wheel['low'] = fuzz.trimf(right_wheel.universe, [0, 0, 1.5])
        right_wheel['high'] = fuzz.trapmf(right_wheel.universe, [1, 2, 4, 4])

        ## Left wheel ##
        left_wheel['zero'] = fuzz.trimf(left_wheel.universe, [0, 0, 0])
        left_wheel['low'] = fuzz.trimf(left_wheel.universe, [0, 0, 1.5])
        left_wheel['high'] = fuzz.trapmf(left_wheel.universe, [1, 2, 4, 4])

        # Rules
        rules = []

        # Stop
        rules.append(ctrl.Rule(distance['none'], [right_wheel['zero'],  left_wheel['zero']]))

        # Turn
        rules.append(ctrl.Rule((distance['big'] | distance['medium']) & angular_distance['smaller from left'] & (~angular_distance['small from left']), [right_wheel['high'], left_wheel['low']] ))

        rules.append(ctrl.Rule((distance['big'] | distance['medium']) & angular_distance['smaller from right'] & (~angular_distance['small from right']), [right_wheel['low'],  left_wheel['high']]))

        rules.append(ctrl.Rule(angular_distance['smaller from left'], [right_wheel['low'], left_wheel['zero']]))

        rules.append(ctrl.Rule(angular_distance['smaller from right'], [right_wheel['zero'], left_wheel['low']]))

        # Follow
        rules.append(ctrl.Rule(angular_distance['small from right'] | angular_distance['small from left'], [right_wheel['high'], left_wheel['high']]))

        rules.append(ctrl.Rule(distance['small'] & (angular_distance['small from right'] | angular_distance['small from left']), [right_wheel['low'], left_wheel['low']]))

         # Control and simulation
        self.goToGoal_ctrl = ctrl.ControlSystem(rules)
        self.goToGoal_sim = ctrl.ControlSystemSimulation(self.goToGoal_ctrl)

    def get_fuzzy_control(self, dist, ang_dist):
        # Stop
        if (dist <= 0.1):
            return [0, 0, ]
        else:
            self.goToGoal_sim.input['distance'] = dist
            self.goToGoal_sim.input['angular distance'] = ang_dist
            self.goToGoal_sim.compute()
            return [self.goToGoal_sim.output['right wheel speed'], self.goToGoal_sim.output['left wheel speed'],]

def cicle(ang):
    return (ang + 2 * math.pi) % (2*math.pi)

def get_distance_from_goal(a,b):
    return np.linalg.norm(a-b)

def get_angular_distance(v1,v2):

    v1_aux = v1 / np.linalg.norm(v1)
    v2_aux = v2 / np.linalg.norm(v2)

    theta = np.arccos(np.clip(np.dot(v1_aux, v2_aux), -1.0, 1.0))
    rot_theta = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
    theta = theta*180.0/math.pi
    theta = (360.0+theta)%360.0

    if abs(np.dot(np.dot(rot_theta,v2_aux),v1_aux)-1.0) > 1e-2:
        theta = 360.0 - theta

    return theta

def unit_vector(vector):
    return vector / np.linalg.norm(vector)


### Avoid Obstacles com Fuzzy
Nessa célula as técnicas para evitar colisão com obstáculos são implementadas. Além de colisão, é indesejável que o robô fique imobilizado por objetos do cenário.

Utilizamos a o controlador da biblioteca SciKit Fuzzy para gerenciar as regras no ambiente fuzzy. As entradas do modelo fuzzy são dados como Antecedentes (ou *Antecedent*) e as saídas como Consequentes (ou *Consequent*). São criadas regras para associar os Antecedentes com os Consequentes.

Existem três dados necessários como **Antecedentes**, que são as medições dos três sensores frontais, um frontal esquerdo, um frontral direito e um último frontal central. A medição para o sensor frontal utilizada é o mínimo dos dois sensores frontais do robô Pioneer 3-DX. Esses dados são então classificados em dois conjuntos fuzzy: **perto** (ou *near*) e **longe** (ou *far*).

Existem duas informações dadas como saída do modelo fuzzy através dos **Consequentes**: a velocidade e a direção. O dado da velocidade pertence aos conjuntos **parado** (ou *no movement*), **lento** (ou *slow*) e **rápido** (ou *fast*). A direção pode pertencer aos conjuntos **esquerda** (ou *left*), **frente** (ou *forward*) ou **direita** (ou *right*).

Com as informações obtidas pelos sensores, o robô decide qual a melhor ação a tomar para evitar obstáculos. O conjunto de regras usadas para isso são exibidos na tabela a seguir:

| Regra |Sensor Esquerdo|Sensor Frontal|Sensor Direito|Velocidade|Direção|
|-------|---------------|--------------|--------------|----------|-------|
|1|perto|perto|perto|parado|esquerda|
|2|longe|perto|longe|lento|frente|
|3|perto|longe|perto|lento|frente|
|4|longe|longe|longe|rápido|frente|
|5|perto|perto|longe|lento|esquerda|
|6|longe|perto|perto|lento|direita|
|7|perto|longe|longe|rápido|frente|
|8|longe|longe|perto|rápido|frente|



In [6]:
ls = ctrl.Antecedent(np.arange(0,10.1,0.1), 'left sensor')
fs = ctrl.Antecedent(np.arange(0,10.1,0.1), 'front sensors')
rs = ctrl.Antecedent(np.arange(0,10.1,0.1), 'right sensor')

ls['near'] = fuzz.trimf(ls.universe, [0, 0, 3])
ls['far'] = fuzz.trapmf(ls.universe, [0, 3, 10, 10])

fs['near'] = fuzz.trimf(fs.universe, [0, 0, 3])
fs['far'] = fuzz.trapmf(ls.universe, [0, 3, 10, 10])

rs['near'] = fuzz.trimf(rs.universe, [0, 0, 3])
rs['far'] = fuzz.trapmf(ls.universe, [0, 3, 10, 10])

f_vel = ctrl.Consequent(np.arange(0, 11, 1), 'velocity')
f_angle = ctrl.Consequent(np.arange(-2,2,0.1), 'angle')

f_vel['no movement'] = fuzz.trimf(f_vel.universe, [0, 0, 3])
f_vel['slow'] = fuzz.trimf(f_vel.universe, [0, 3, 6])
f_vel['fast'] = fuzz.trapmf(f_vel.universe, [3, 6, 10, 10])

f_angle['left'] = fuzz.trimf(f_angle.universe, [-2, -2, 0])
f_angle['forward'] = fuzz.trimf(f_angle.universe, [-0.2, 0, 0.2])
f_angle['right'] = fuzz.trimf(f_angle.universe, [0, 2, 2])

rules = []
rules.append(ctrl.Rule(ls['near'] & fs['near'] & rs['near'], [f_vel['no movement'], f_angle['left']]))
rules.append(ctrl.Rule(ls['far'] & fs['near'] & rs['far'], [f_vel['slow'], f_angle['forward']]))
rules.append(ctrl.Rule(ls['near'] & fs['far'] & rs['near'], [f_vel['slow'], f_angle['forward']]))
rules.append(ctrl.Rule(ls['far'] & fs['far'] & rs['far'], [f_vel['fast'], f_angle['forward']]))

rules.append(ctrl.Rule(ls['near'] & fs['near'] & rs['far'], [f_vel['slow'], f_angle['left']]))
rules.append(ctrl.Rule(ls['far'] & fs['near'] & rs['near'], [f_vel['slow'], f_angle['right']]))

rules.append(ctrl.Rule(ls['near'] & fs['far'] & rs['far'], [f_vel['fast'], f_angle['forward']]))
rules.append(ctrl.Rule(ls['far'] & fs['far'] & rs['near'], [f_vel['fast'], f_angle['forward']]))

def avoid_obstacles(sensors):
    avoid_ctrl = ctrl.ControlSystem(rules)
    avoid = ctrl.ControlSystemSimulation(avoid_ctrl)
    
    avoid.input['left sensor'] = sensors[2]
    avoid.input['front sensors'] = min(sensors[3], sensors[4])
    avoid.input['right sensor'] = sensors[5]
    
    avoid.compute()
    
    vel = avoid.output['velocity']
    angle = avoid.output['angle']
    
#    return [(1 + min(0, angle)) * vel, (1 - max(0, angle)) * vel]
    return [(1 - max(0, angle)) * vel, (1 + min(0, angle)) * vel]
    

In [7]:
noDetectionDist = 0.7
defaultVelocity = 1
noDetectionDistDiag = 0.4

### Loop Avoid Obstacle com Fuzzy

In [8]:
robot = Robot()

while(robot.get_connection_status() != -1):
    us_distances = robot.read_ultrassonic_sensors()
    (left_vel, right_vel) = avoid_obstacles(us_distances)
    
    robot.set_left_velocity(left_vel)
    robot.set_right_velocity(right_vel)

'robot = Robot()\n\nwhile(robot.get_connection_status() != -1):\n    us_distances = robot.read_ultrassonic_sensors()\n    (left_vel, right_vel) = avoid_obstacles(us_distances)\n    \n    robot.set_left_velocity(left_vel)\n    robot.set_right_velocity(right_vel)\n'

### Loop Go to Goal com PID

In [9]:
robot = Robot()

limit = 2*math.pi
lastTime=time.time()
clockwiseSpinL = False
clockwiseSpinR = False
lastEncoderL = robot.get_joint_position('left')
lastEncoderR = robot.get_joint_position('right')

position = robot.get_current_position()
orientation = robot.get_current_orientation()

alpha = orientation[2]
x = position[0]
y = position[1]


goal = np.array([3.32, -1.8])
goToGoalPID = GoToGoalPID()

path = []
path_odometry = []
while(robot.get_connection_status() != -1):
    angle = robot.get_current_orientation()[2]
    position = robot.get_current_position()    
    
    vGoToPID = goToGoalPID.go_to_goal_PID(position, angle, goal)
    
    robot.set_left_velocity(vGoToPID[1])
    robot.set_right_velocity(vGoToPID[0])

    # Odometria
    x, y, alpha = new_position(alpha, x, y)
    path.append([-position[1], position[0]])
    path_odometry.append([-y, x])


"robot = Robot()\n\nlimit = 2*math.pi\nlastTime=time.time()\nclockwiseSpinL = False\nclockwiseSpinR = False\nlastEncoderL = robot.get_joint_position('left')\nlastEncoderR = robot.get_joint_position('right')\n\nposition = robot.get_current_position()\norientation = robot.get_current_orientation()\n\nalpha = orientation[2]\nx = position[0]\ny = position[1]\n\n\ngoal = np.array([3.32, -1.8])\ngoToGoalPID = GoToGoalPID()\n\npath = []\npath_odometry = []\nwhile(robot.get_connection_status() != -1):\n    angle = robot.get_current_orientation()[2]\n    position = robot.get_current_position()    \n    \n    vGoToPID = goToGoalPID.go_to_goal_PID(position, angle, goal)\n    \n    robot.set_left_velocity(vGoToPID[1])\n    robot.set_right_velocity(vGoToPID[0])\n\n    # Odometria\n    x, y, alpha = new_position(alpha, x, y)\n    path.append([-position[1], position[0]])\n    path_odometry.append([-y, x])\n\n"

### Loop Wall Follow com Fuzzy

In [10]:
robot = Robot()

defaultVelocity = 1
minDist = 0.0
maxDist = 6.0
interval = 0.1

while (robot.get_connection_status() != -1):
    us_distances = robot.read_ultrassonic_sensors()
    vel = followWall(us_distances[5], us_distances[7])
    
    robot.set_left_velocity(vel[0] * 4)
    robot.set_right_velocity(vel[1] * 4)


'\nrobot = Robot()\n\ndefaultVelocity = 1\nminDist = 0.0\nmaxDist = 6.0\ninterval = 0.1\n\nwhile (robot.get_connection_status() != -1):\n    us_distances = robot.read_ultrassonic_sensors()\n    vel = followWall(us_distances[5], us_distances[7])\n    \n    robot.set_left_velocity(vel[0] * 4)\n    robot.set_right_velocity(vel[1] * 4)\n'

### Loop Go to Goal com Fuzzy

In [11]:
robot = Robot()

goal = np.array([2.77, -2.92])
goToGoalFuzzy = GoToGoalFuzzy()

while(robot.get_connection_status() != -1):
    # Include the goal
    
    current_position = np.array(robot.get_current_position()[:-1])
    current_angle = cicle(robot.get_current_orientation()[2])
    distance = get_distance_from_goal(current_position, goal)
    angle_distance = get_angular_distance(goal - current_position, np.array([math.cos(current_angle),math.sin(current_angle)])) # graus

    vel = goToGoalFuzzy.get_fuzzy_control(distance, angle_distance)
    robot.set_right_velocity(vel[0])
    robot.set_left_velocity(vel[1])


'robot = Robot()\n\ngoal = np.array([2.77, -2.92])\ngoToGoalFuzzy = GoToGoalFuzzy()\n\nwhile(robot.get_connection_status() != -1):\n    # Include the goal\n    \n    current_position = np.array(robot.get_current_position()[:-1])\n    current_angle = cicle(robot.get_current_orientation()[2])\n    distance = get_distance_from_goal(current_position, goal)\n    angle_distance = get_angular_distance(goal - current_position, np.array([math.cos(current_angle),math.sin(current_angle)])) # graus\n\n    vel = goToGoalFuzzy.get_fuzzy_control(distance, angle_distance)\n    robot.set_right_velocity(vel[0])\n    robot.set_left_velocity(vel[1])\n'

### Loop Go to Goal mais Avoid Obstacle com Fuzzy

O algoritmo nessa célula faz o robô se movimentar até o ponto alvo evitando obstáculos no meio do caminho. Para isso as técnicas de Go to Goal e Avoid Obstacle são integradas, fazendo que o robô se aproxime do destinho sempre que não houver obstáculos próximos e os evite caso contrário.

In [12]:
robot = Robot()

goal = np.array([2.77, 2])
goToGoalFuzzy = GoToGoalFuzzy()

position = robot.get_current_position()
orientation = robot.get_current_orientation()

while(robot.get_connection_status() != -1):
    
    current_position = np.array(robot.get_current_position()[:-1])
    current_angle = cicle(robot.get_current_orientation()[2])
    distance = get_distance_from_goal(current_position, goal)
    angle_distance = get_angular_distance(goal - current_position, np.array([math.cos(current_angle),math.sin(current_angle)])) # graus

    vel = goToGoalFuzzy.get_fuzzy_control(distance, angle_distance)
    
    us_distances = robot.read_ultrassonic_sensors()

    vel_ = avoid_obstacles(us_distances)
    
    flag = False
    for i in us_distances[:8]:
        flag = flag or (i < noDetectionDist)
    
    if (flag):
        robot.set_left_velocity(vel_[0])
        robot.set_right_velocity(vel_[1])
    else:
        robot.set_left_velocity(vel[1] * 2)
        robot.set_right_velocity(vel[0] * 2)



Connected to remoteApi server.
 Pioneer_p3dx_ultrasonicSensor1 connected.
 Pioneer_p3dx_ultrasonicSensor2 connected.
 Pioneer_p3dx_ultrasonicSensor3 connected.
 Pioneer_p3dx_ultrasonicSensor4 connected.
 Pioneer_p3dx_ultrasonicSensor5 connected.
 Pioneer_p3dx_ultrasonicSensor6 connected.
 Pioneer_p3dx_ultrasonicSensor7 connected.
 Pioneer_p3dx_ultrasonicSensor8 connected.
 Pioneer_p3dx_ultrasonicSensor9 connected.
 Pioneer_p3dx_ultrasonicSensor10 connected.
 Pioneer_p3dx_ultrasonicSensor11 connected.
 Pioneer_p3dx_ultrasonicSensor12 connected.
 Pioneer_p3dx_ultrasonicSensor13 connected.
 Pioneer_p3dx_ultrasonicSensor14 connected.
 Pioneer_p3dx_ultrasonicSensor15 connected.
 Pioneer_p3dx_ultrasonicSensor16 connected.
 Vision sensor connected.
 Laser connected.
 Left motor connected.
 Right motor connected.
 Robot connected.


In [13]:
'''if(len(path) > 0):
    path = np.array(path)
    path_odometry = np.array(path_odometry)

    difx = path[1,0] - path_odometry[1,0]
    dify = path[1,1] - path_odometry[1,1]
    path_odometry[:,0] = path_odometry[:,0]+difx
    path_odometry[:,1] = path_odometry[:,1]+dify

    plot_odometry_vs_GT(path, path_odometry)'''

'if(len(path) > 0):\n    path = np.array(path)\n    path_odometry = np.array(path_odometry)\n\n    difx = path[1,0] - path_odometry[1,0]\n    dify = path[1,1] - path_odometry[1,1]\n    path_odometry[:,0] = path_odometry[:,0]+difx\n    path_odometry[:,1] = path_odometry[:,1]+dify\n\n    plot_odometry_vs_GT(path, path_odometry)'

## Resultados

O vídeo a seguir (disponível em https://github.com/pedroonop/MO651/blob/master/P2/videos/FullVideo.mp4) mostra a simulação com a aplicação de cada um dos métodos mostrados. Primeiramente o vídeo mostra o robô seguindo em direção a um ponto de destino (vaso com plantas) usando as técnicas de Go to Goal com PID e o mesmo é feito usando técnicas de Go to Goal com lógica Fuzzy. Em seguida mostra o robô explorando o ambiente usando técnicas de Wall Follow com lógica Fuzzy. Depois mostra o robô explorando o ambiente evitando obstáculos usabdo técnicas de Avoid Obstacle com lógica Fuzzy. Por fim o vídeo mostra o robô indo para um determinado ponto do cenário evitando obstáculo usando técnicas de Go to Goal e Avoid Obstacle.

Nota-se que o robô consegue chegar no ponto objetivo nos experimentos com as técnicas de *Go to Goal* executadas. Quando aplicado o algoritmo de *Wall Follow* o robô contorna toda a sala na qual está inserido, seguindo a parede. Na simulação com Avoid Obstacle o robô explora o ambiente sempre evitando obstáculos e escolhendo o melhor caminho para virar e seguir em frente. Ademais, quando simulado o algoritmo de alcançar um ponto objetivo com obstáculos no caminho, o robô evita os obstáculos e consegue alcançar o ponto objetivo.

<video controls src="../videos/FullVideo.mp4" style="width: 800px;">


Comparamos as técnicas do Go to Goal usando as informações de Ground Truth e a posição e orientação cálculados usando Odometria. As imagens a seguir mostram as rotas tomadas pelo robô em cada cenário. A rota em azul é aquela usando Ground Truth e em vermelho usando Odometria.

<img src="../imgs/gtg+ao.jpg">

<img src="../imgs/gtg+ao_.png">

Em ambos os casos os pontos finais alcançados pelo robô coincidem com o ponto objetivo, mesmo existindo obstáculos no caminho. 

## Conclusão

Neste trabalho implementamos com sucesso técnicas de *Avoid Obstacles*, *Wall Follow* e *Go to Goal* (até mesmo usando cálculos de Odometria para obter localização e orientação do robô) para o ambiente simulado.